In [1]:
#default_exp oo_hp

In [2]:
#export
import fastai2
from fastai2.tabular.all import *
from fastai2.metrics import *

In [3]:
#export
from copy import deepcopy

In [4]:
#export
import optuna

In [5]:
#export
SEED = 42

In [6]:
from nbdev import export

In [7]:
from nbdev.export import *

In [8]:
#export
from collections.abc import Iterable

# HyPSTER Classes

In [9]:
#export
from inspect import signature, Parameter
import functools

def auto_assign(func):
    # Signature:
    sig = signature(func)
    for name, param in sig.parameters.items():
        if param.kind in (Parameter.VAR_POSITIONAL, Parameter.VAR_KEYWORD):
            raise RuntimeError('Unable to auto assign if *args or **kwargs in signature.')
    # Wrapper:
    @functools.wraps(func)
    def wrapper(self, *args, **kwargs):
        for i, (name, param) in enumerate(sig.parameters.items()):
            # Skip 'self' param:
            if i == 0: continue
            # Search value in args, kwargs or defaults:
            if i - 1 < len(args):
                val = args[i - 1]
            elif name in kwargs:
                val = kwargs[name]
            else:
                val = param.default
            setattr(self, name, val)
        func(self, *args, **kwargs)
    return wrapper

In [10]:
#export
class HypsterBase(object):
    def __init__(self): return
#TODO: add stuff here

In [11]:
#export
class HpExpression(object):
    def __init__(self, exp1, exp2):
        self.exp1 = exp1; self.exp2 = exp2    
    
    def sample(self, trial): raise NotImplementedError
    
    def get_name(self):
        if self.name is not None: return self.name

        name = ""
        if self.exp1 is not None and isinstance(self.exp1, HpExpression) and hasattr(self.exp1, "name"):
            name += self.exp1.name
        if self.exp2 is not None and isinstance(self.exp2, HpExpression) and hasattr(self.exp2, "name"):
            if len(name) > 0:
                name +=  "_"
            name += self.exp2.name
        self.name = name
        return self.name
        #TODO: refactor

    def __add__(self, other):  return AddExpression(self, other)
    def __radd__(self, other): return AddExpression(other, self)
    def __sub__(self, other):  return SubExpression(self, other)
    def __rsub__(self, other): return SubExpression(other, self)
    def __mul__(self, other):  return MulExpression(self, other)
    def __rmul__(self, other): return MulExpression(other, self)
    def __div__(self, other):  return DivExpression(self, other)
    def __rdiv__(self, other): return DivExpression(other, self)
    def __pow__(self, other):  return PowExpression(self, other)
    def __rpow__(self, other): return PowExpression(other, self)

In [12]:
#export
HYPSTER_TYPES = (HypsterBase, HpExpression)

In [13]:
#export
class SubExpression(HpExpression):
    def sample(self, trial):
        exp1 = sample_hp(self.exp1, trial)
        exp2 = sample_hp(self.exp2, trial)
        return exp1 - exp2

In [14]:
#export
class AddExpression(HpExpression):    
    def sample(self, trial):
        exp1 = sample_hp(self.exp1, trial)
        exp2 = sample_hp(self.exp2, trial)
        return exp1 + exp2

In [15]:
#export
class MulExpression(HpExpression):    
    def sample(self, trial):
        exp1 = sample_hp(self.exp1, trial)
        exp2 = sample_hp(self.exp2, trial)
        return exp1 * exp2

In [16]:
#export
class DivExpression(HpExpression):    
    def sample(self, trial):
        exp1 = sample_hp(self.exp1, trial)
        exp2 = sample_hp(self.exp2, trial)
        return exp1 / exp2

In [17]:
#export
class PowExpression(HpExpression):    
    def sample(self, trial):
        exp1 = sample_hp(self.exp1, trial)
        exp2 = sample_hp(self.exp2, trial)
        return exp1 ** exp2

In [18]:
#TODO: add round, int, floor etc...
#TODO: add Brackets (), [] etc...?

In [19]:
#export
class HpInt(HpExpression):
    @auto_assign
    def __init__(self, name, low, high, step=1):
        self.result = None
    
    def sample(self, trial):
        self.result = trial.suggest_int(self.name, self.low, self.high, self.step)
        return self.result

In [20]:
#export
class HpFloat(HpExpression):
    @auto_assign
    def __init__(self, name, low, high, log=False, step=None):
        self.result = None
        #TODO: check what's up with log and step
        #TODO: move result to HpExpression?
        
    def sample(self, trial): 
        #self.result = ifnone(self.result, trial.suggest_float(self.name, self.low, self.high))
        self.result = trial.suggest_float(self.name, self.low, self.high)
        return self.result
    
    #TODO: warn if log=True & step is not None
    #TODO: check what is the "*" in the function definition

In [21]:
#export
def _log_optuna_param(param_name, result, trial):
    trial.set_user_attr(param_name, result)

In [22]:
#export
class HpFunc(HpExpression):
    def __init__(self, name, func, **kwargs):
        self.name = name
        self.func = func
        self.kwargs = kwargs
        self.result = None
    
    def sample(self, trial):
        self.result = self.func(trial, **self.kwargs)
        _log_optuna_param(self.name, self.result, trial)
        return self.result

In [23]:
#export
class HpCategorical(HpExpression):
    @auto_assign
    def __init__(self, name, choices): 
        self.result = None
    
    def sample(self, trial): 
        #print(f"result = {self.result}")
        #if self.result is not None:
        #    return self.result
        
        choices           = self.choices
        name              = self.name
        optuna_valid_cats = [str, int, float, bool] #TODO: add more + move to global area
        
        if any([type(choice) not in optuna_valid_cats for choice in self.choices]):
            #TODO: add check for "choice.__name__"
            self.items_str = [choice.__name__ for choice in choices]
            self.str_dict  = dict(zip(self.items_str, choices))
            chosen_hp      = trial.suggest_categorical(name, self.str_dict)
            self.result    = self.str_dict[chosen_hp]
        else:
            self.result = trial.suggest_categorical(name, choices)
        return self.result

In [24]:
#export
class HpVarLenList(HpExpression):
    #TODO: think of a better name?
    @auto_assign
    def __init__(self, name, min_len, max_len, hp, same_value=False): pass     
    
    def sample(self, trial):
        lst_len = trial.suggest_int(self.name, self.min_len, self.max_len)
        lst = []
        if (self.same_value) or (not contains_hypster(self.hp, HYPSTER_TYPES)):
            lst = [sample_hp(self.hp, trial)] * lst_len
        else:
            for i in range(lst_len):
                hp = deepcopy(self.hp)
                hp.result = None
                hp.name = f"{hp.get_name()}_{i+1}"
                result = sample_hp(hp, trial)
                lst.append(result)
                #TODO keep self.result?
        return lst

In [25]:
#TODO: make HpVarLenTuple

In [26]:
#export
def list_to_tuple(lst): return (*lst, )

In [27]:
#export
def populate_iterable(iterable, trial):
    sampled_lst = []
    for item in iterable:
        if isinstance(item, HpToggle):
            if sample_hp(item, trial):
                sampled_lst.append(sample_hp(item.hp, trial))
        else:
            sampled_lst.append(sample_hp(item, trial))
    return sampled_lst

In [28]:
#export
def populate_dict(dct, trial):
    sampled_dict = {}
    for key, value in dct.items():
        if isinstance(value, HpToggle):
            if sample_hp(value, trial):
                sampled_dict[key] = sample_hp(value.hp, trial)
        else:
            sampled_dict[key] = sample_hp(value, trial)
    return sampled_dict

In [29]:
#export
class HpList(HpExpression):
    @auto_assign
    def __init__(self, lst): pass
    
    def sample(self, trial):
        return populate_iterable(self.lst, trial)

In [30]:
#export
class HpTuple(HpExpression):
    @auto_assign
    def __init__(self, tup): pass
    
    def sample(self, trial):
        return list_to_tuple(populate_iterable(self.tup, trial))

In [31]:
#export
class HpDict(HpExpression):
    @auto_assign
    def __init__(self, dct): pass
    
    def sample(self, trial):
        return populate_dict(self.dct, trial)

In [32]:
#export
class HpBool(HpCategorical):
    def __init__(self, name):
        super().__init__(name, choices=[False, True])

In [33]:
#export
class HpToggle(HpBool):
    @auto_assign
    def __init__(self, name, hp): return
    def sample(self, trial): return HpBool(self.name).sample(trial)
    #automatically add name? toggle_ + hp.__name__

# Tests

In [34]:
#export
DATA_STRUCTURES = (set, list, tuple, dict)

In [35]:
#export
def contains_hypster(x, types):
    if not isinstance(x, DATA_STRUCTURES):
        x = [x]
    elif isinstance(x, dict):
        x = x.values()
        
    for item in x:
        if isinstance(item, DATA_STRUCTURES):
            if contains_hypster(item, types):
                return True
        else:
            if isinstance(item, types):
                return True
    return False

In [36]:
#TODO!: check if class attributes / methods? have hypster in them

## Test contains_hypster

In [37]:
hps = []

In [38]:
hps.append(HpInt("hi!", 2, 10))

In [39]:
hps.append({"hi" : 4, "hello": HpFloat("sdf", 1.0, 2.0)})

In [40]:
hps.append(["a", 2, HpInt("hola", 1, 19)])

In [41]:
hps.append(["a", 2, {"ho": "hello"}, {"hi" : HpCategorical("d", [1,2,3])}])

In [42]:
[contains_hypster(hp, HpExpression) for hp in hps]

[True, True, True, True]

In [43]:
#export
def sample_hp(hp, trial):
    if not contains_hypster(hp, HYPSTER_TYPES):
        return hp
    #TODO: change to dynamic dispatch
    if isinstance(hp, list):
        hp = HpList(hp)
    elif isinstance(hp, tuple):
        hp = HpTuple(hp)
    elif isinstance(hp, dict):
        hp = HpDict(hp)
    return hp.sample(trial)
    #TODO: handle names
    #TODO: handle list of lists
#TODO!: check if class attributes / methods? have hypster in them

In [44]:
#export
def run_hp_test(hp):
    def objective(trial):
        print(sample_hp(hp, trial))
        return 1.0        
    
    optuna.logging.set_verbosity(0)
    pruner = optuna.pruners.NopPruner()
    study = optuna.create_study(direction="maximize", pruner=pruner)
    study.optimize(objective, n_trials=5, timeout=600)

### HpToggle

In [45]:
x = [1, 5, HpToggle("10_toggle", 10)]

In [46]:
run_hp_test(x)

[1, 5, 10]
[1, 5]
[1, 5]
[1, 5, 10]
[1, 5]


In [47]:
y = (1, 5, HpToggle("10_toggle", 10))

In [48]:
run_hp_test(y)

(1, 5, 10)
(1, 5, 10)
(1, 5, 10)
(1, 5)
(1, 5)


In [49]:
z = {"hi" : 1, "hello" : 5, "howdy" : HpToggle("10_toggle", 10)}

In [50]:
run_hp_test(z)

{'hi': 1, 'hello': 5, 'howdy': 10}
{'hi': 1, 'hello': 5}
{'hi': 1, 'hello': 5}
{'hi': 1, 'hello': 5}
{'hi': 1, 'hello': 5}


### Int

In [51]:
x = HpInt("start_mom", 2, 10)
y = HpInt("layer_size", 50, 300, 50)
z = HpVarLenList("n_layers", 1, 5, x, same_value=False)

In [52]:
run_hp_test(x)

5
7
3
2
6


In [53]:
run_hp_test(y)

150
300
300
50
50


In [54]:
run_hp_test(z)

[6, 7]
[9, 4]
[3]
[6, 2, 10]
[8]


### Float

In [55]:
x = HpFloat("start_mom", 1.0, 2.0)
y = HpVarLenList("n_layers", 1, 5, x, same_value=False)

In [56]:
run_hp_test(x)

1.3776474238625527
1.930702334284998
1.5475101416558412
1.9452080511363525
1.211928605620524


In [57]:
run_hp_test(y)

[1.701810921597919, 1.7189529772314946, 1.0753404694482975]
[1.798616017550727, 1.884694497675862, 1.6396319577680487, 1.3195760812244925, 1.6286621780215005]
[1.4704952296041163, 1.2777424289398476, 1.3584374814355618]
[1.2894135225624106, 1.7206882462982516, 1.8171083405363968, 1.639067359651923, 1.9004681349684747]
[1.101503831213158]


### Categorical

In [58]:
x = HpCategorical("cats!", ["cat", "meow", "rrrrrr"])
y = HpCategorical("cats!", [Adam, SGD, QHAdam])
z = HpVarLenList("n_layers", 1, 5, x, same_value=False)

In [59]:
run_hp_test(x)

cat
rrrrrr
meow
rrrrrr
meow


In [60]:
run_hp_test(y)

<function SGD at 0x000001B9C849F950>
<function SGD at 0x000001B9C849F950>
<function QHAdam at 0x000001B9C84A4158>
<function SGD at 0x000001B9C849F950>
<function QHAdam at 0x000001B9C84A4158>


In [61]:
run_hp_test(z)

['rrrrrr', 'meow']
['rrrrrr', 'rrrrrr']
['meow']
['cat', 'rrrrrr', 'cat', 'rrrrrr']
['meow', 'meow', 'rrrrrr', 'meow']


### Boolean

In [62]:
x = HpBool("booli!")
y = HpVarLenList("n_layers", 1, 5, x, same_value=False)

In [63]:
run_hp_test(x)

False
True
True
False
False


In [64]:
run_hp_test(y)

[True, True, False, True, True]
[False, False, True, False]
[True, False, True, False, False]
[True, True, False, False]
[False]


### HpIterable

In [65]:
x = HpBool("boolean")

In [66]:
lst = [x, 5]
tup = [x, 5]
dct = {"first" : x, "second" : 5}

In [67]:
start_mom = HpFloat("start_mom", 0.85, 0.99)

In [68]:
mom = (start_mom, start_mom - 0.1)

In [69]:
run_hp_test(mom)

(0.8603043184889482, 0.7603043184889482)
(0.9170521195365139, 0.8170521195365139)
(0.8927897536933024, 0.7927897536933024)
(0.8806939455883946, 0.7806939455883947)
(0.8608808408713594, 0.7608808408713594)


In [70]:
run_hp_test(lst)

[True, 5]
[False, 5]
[False, 5]
[True, 5]
[False, 5]


In [71]:
run_hp_test(tup)

[True, 5]
[False, 5]
[True, 5]
[False, 5]
[False, 5]


In [72]:
run_hp_test(dct)

{'first': True, 'second': 5}
{'first': True, 'second': 5}
{'first': False, 'second': 5}
{'first': True, 'second': 5}
{'first': True, 'second': 5}


In [73]:
#TODO consider adding "name" into HpExpression

In [74]:
#HpFunc

In [75]:
#TODO: support expressions like HpInt(...) * HpFloat(...) ?

In [76]:
notebook2script()

Converted 00_core.ipynb.
Converted 01_api.ipynb.
Converted 02_oo_hp.ipynb.
Converted 03_hypster_prepare.ipynb.
Converted 04_tabular_api.ipynb.
Converted 05_sklearn.ipynb.
Converted fastai_adult_tutorial.ipynb.
Converted index.ipynb.
